In [1]:
import gmaps
import requests, json 
import time

with open('apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
    
gmaps.configure(api_key=api_key)

In [2]:
new_york_coordinates = (40.75, -74.00)
gmaps.figure(center=new_york_coordinates, zoom_level=12)


Figure(layout=FigureLayout(height='420px'))

In [3]:
ratings = {}
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# union st and 4th ave
location ='40.677485,-73.983310'
rankby='distance'
keyword='pizza'

r = requests.get(url + 
                 '?key=' + api_key + 
                 '&location=' + location +
                 '&rankby=' + rankby + 
                 '&keyword=' + keyword
                ) 

j = r.json()
results = j['results'] 

for res in results: 
    ratings[res['name']] = res['rating']
    print(res['name'], res['rating'])

while 'next_page_token' in j:
    next_page_token = j['next_page_token']
    time.sleep(5)
    r = requests.get(url + '?pagetoken=' + next_page_token +
                     '&key=' + api_key)
    j = r.json()
    results = j['results'] 
    for res in results: 
        ratings[res['name']] = res['rating']
        print(res['name'], res['rating'])

for name, rating in reversed(sorted(ratings.items(), key=lambda item: item[1])):
    print (rating, name)


Union Street Pizza 4.5
LBK pizza 4.6
Cottabene Well Done Pizza Corporation 4.7
Pizza Cotta Bene 4.4
La Villa Pizzeria 4.5
Pizza Town 4.5
Pizza Secret 4.5
Mother Dough Pizza 4.5
Pizza Superstar - Dean St 3.6
Gino's Pizzeria 4.4
Roma Pizza 4.4
Patsy’s Pizzeria 4.5
Bergen Pizza 4.4
Brooklyn Pizza Market 4.1
Giardini Pizza 4.1
Smiling Pizzeria 4.2
Caruso's 3.7
Sal's Pizza Store 4.4
Numero 28 4.4
Via Roma Pizza Bar 4.3
Monte's 4.4
Papa John's Pizza 3.5
Brooklyn Pizza 5
The Best Joe's Pizza of Park Slope 3.7
Numero 28 Pizzeria 4.2
Pino's La Forchetta Pizza 4.2
Mario's Pizzeria 3.6
Table 87 Gowanus 4.6
Angelica 3.8
Amorina Cucina Rustica 4.4
Not Ray's Pizza 4.3
Lean Crust Pizza 4.4
Sottocasa 4.6
Pizza Plus 4.3
Layla Jones 4.3
Brooklyn's Homeslice Pizza 4.5
Forcella Fried Pizza 4.5
Pizza Moto 4.7
Lucali 4.5
Francesco's Pizzeria & Trattoria 4.2
L'Arte Della Pizza Brooklyn 4.7
Two Boots Park Slope 4.3
Crown Fried Chicken & Pizza 3.7
Peppino's Park Slope 4.6
Joe's Pizza of Park Slope 4
2 Bros. Pi

In [5]:
for name, rating in reversed(sorted(ratings.items(), key=lambda item: item[1])):
    print (rating, name)


5 Brooklyn Pizza
4.7 Baciccia Pizza e Cucina
4.7 L'Arte Della Pizza Brooklyn
4.7 Pizza Moto
4.7 Cottabene Well Done Pizza Corporation
4.6 Luzzo's BK
4.6 The House of Pizza & Calzone
4.6 Piz-zetta Pizzeria
4.6 Peppino's Park Slope
4.6 Sottocasa
4.6 Table 87 Gowanus
4.6 LBK pizza
4.5 Brado Thin Crust Pizza
4.5 Emily - Brooklyn
4.5 Lucali
4.5 Forcella Fried Pizza
4.5 Brooklyn's Homeslice Pizza
4.5 Patsy’s Pizzeria
4.5 Mother Dough Pizza
4.5 Pizza Secret
4.5 Pizza Town
4.5 La Villa Pizzeria
4.5 Union Street Pizza
4.4 Ogliastro Pizza Bar
4.4 My Little Pizzeria
4.4 Lean Crust Pizza
4.4 Amorina Cucina Rustica
4.4 Monte's
4.4 Numero 28
4.4 Sal's Pizza Store
4.4 Bergen Pizza
4.4 Roma Pizza
4.4 Gino's Pizzeria
4.4 Pizza Cotta Bene
4.3 Fresh 99 cents pizza
4.3 Two Boots Park Slope
4.3 Layla Jones
4.3 Pizza Plus
4.3 Not Ray's Pizza
4.3 Mario's Pizzeria
4.3 Via Roma Pizza Bar
4.2 Francesco's Pizzeria & Trattoria
4.2 Pino's La Forchetta Pizza
4.2 Numero 28 Pizzeria
4.2 Smiling Pizzeria
4.1 2 Bros. P

In [4]:
j['results']

[{'geometry': {'location': {'lat': 40.6776064, 'lng': -73.9799572},
   'viewport': {'northeast': {'lat': 40.67890907989272,
     'lng': -73.97848537010728},
    'southwest': {'lat': 40.67620942010727, 'lng': -73.98118502989271}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
  'id': '1fddefd4701b20059c15b7396b6fdef0bf3f2ec6',
  'name': "L'Arte Della Pizza Brooklyn",
  'opening_hours': {'open_now': True},
  'photos': [{'height': 4032,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100627033292864918338">A Google User</a>'],
    'photo_reference': 'CmRZAAAA9-RkeI83s3I9J6ECIIcJHNmCHvnYt_YdI-fi-wk_t89IuuoHxtgJKl-4yPv5AbyAnP_NcNtH0GLaemtS4AlcdDV9fNbqjDpC1MWZE0go_mSqvzx-Wd5Rvl55A-TG-TGOEhBe32M7sN7c0-lfAaIIkP3IGhQv-6pQiVEnD9MIX2kIUiAZTPTLAw',
    'width': 3024}],
  'place_id': 'ChIJ0QUjip5bwokRSu67iFCYRJA',
  'plus_code': {'compound_code': 'M2HC+22 New York',
   'global_code': '87G8M2HC+22'},
  'rating': 4.7,
  'reference': 'ChIJ0Q